In [2]:
import gym
import numpy as np
import random

# 检查Gym版本
import gym
from packaging import version

gym_version = gym.__version__
print(f"当前Gym版本: {gym_version}")

# 初始化FrozenLake环境
# 根据Gym版本选择是否使用gym.make('FrozenLake-v1')或其他方式
env = gym.make('FrozenLake-v1', is_slippery=False)  # is_slippery=False 时环境是确定性的

# 获取状态和动作的数量
state_size = env.observation_space.n
action_size = env.action_space.n

# 初始化Q表为零
Q_table = np.zeros((state_size, action_size))

# 超参数
alpha = 0.8  # 学习率
gamma = 0.95  # 折扣因子
epsilon = 1.0  # 探索率初始值
epsilon_min = 0.01  # 探索率最小值
epsilon_decay = 0.995  # 探索率衰减率
num_episodes = 2000  # 训练回合数
max_steps = 100  # 每回合最大步数

# 用于记录每个回合的奖励
rewards = []

for episode in range(num_episodes):
    # 重置环境并获取初始状态
    reset_result = env.reset()
    if version.parse(gym_version) >= version.parse("0.26"):
        state, _ = reset_result
    else:
        state = reset_result

    total_reward = 0
    done = False

    for step in range(max_steps):
        # 决策：使用ε-贪婪策略选择动作
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # 随机探索
        else:
            action = np.argmax(Q_table[state, :])  # 利用已学知识

        # 执行动作，获得下一个状态、奖励和是否完成
        step_result = env.step(action)
        if version.parse(gym_version) >= version.parse("0.26"):
            next_state, reward, terminated, truncated, _ = step_result
            done = terminated or truncated
        else:
            next_state, reward, done, _ = step_result

        # 更新Q表
        old_value = Q_table[state, action]
        next_max = np.max(Q_table[next_state, :])
        new_value = old_value + alpha * (reward + gamma * next_max - old_value)
        Q_table[state, action] = new_value

        # 累积奖励
        total_reward += reward

        # 转移到下一个状态
        state = next_state

        if done:
            break

    # 探索率衰减
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    rewards.append(total_reward)

    # 每100回合打印一次进度
    if (episode + 1) % 100 == 0:
        avg_reward = sum(rewards[-100:]) / 100
        print(f"回合 {episode + 1}/{num_episodes} - 平均奖励: {avg_reward:.2f} - 探索率: {epsilon:.2f}")

print("\n训练完成！\n")

# 测试智能体
num_test_episodes = 100
successes = 0

for episode in range(num_test_episodes):
    reset_result = env.reset()
    if version.parse(gym_version) >= version.parse("0.26"):
        state, _ = reset_result
    else:
        state = reset_result

    done = False
    step = 0

    for step in range(max_steps):
        action = np.argmax(Q_table[state, :])  # 选择最优动作
        step_result = env.step(action)
        if version.parse(gym_version) >= version.parse("0.26"):
            next_state, reward, terminated, truncated, _ = step_result
            done = terminated or truncated
        else:
            next_state, reward, done, _ = step_result
        state = next_state

        if done:
            successes += reward
            break

print(f"在 {num_test_episodes} 个测试回合中，成功次数: {int(successes)}")
print(f"成功率: {successes / num_test_episodes * 100}%")

# 打印最终的Q表
print("\n最终的Q表:")
print(Q_table)


当前Gym版本: 0.26.2
回合 100/2000 - 平均奖励: 0.00 - 探索率: 0.61
回合 200/2000 - 平均奖励: 0.01 - 探索率: 0.37


/Users/mikeshinoda/Library/Caches/pypoetry/virtualenvs/ai-algo-ozOH1ZQc-py3.12/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


回合 300/2000 - 平均奖励: 0.01 - 探索率: 0.22
回合 400/2000 - 平均奖励: 0.76 - 探索率: 0.13
回合 500/2000 - 平均奖励: 0.88 - 探索率: 0.08
回合 600/2000 - 平均奖励: 0.95 - 探索率: 0.05
回合 700/2000 - 平均奖励: 0.97 - 探索率: 0.03
回合 800/2000 - 平均奖励: 1.00 - 探索率: 0.02
回合 900/2000 - 平均奖励: 0.98 - 探索率: 0.01
回合 1000/2000 - 平均奖励: 0.99 - 探索率: 0.01
回合 1100/2000 - 平均奖励: 0.99 - 探索率: 0.01
回合 1200/2000 - 平均奖励: 1.00 - 探索率: 0.01
回合 1300/2000 - 平均奖励: 1.00 - 探索率: 0.01
回合 1400/2000 - 平均奖励: 0.99 - 探索率: 0.01
回合 1500/2000 - 平均奖励: 0.99 - 探索率: 0.01
回合 1600/2000 - 平均奖励: 0.98 - 探索率: 0.01
回合 1700/2000 - 平均奖励: 0.99 - 探索率: 0.01
回合 1800/2000 - 平均奖励: 1.00 - 探索率: 0.01
回合 1900/2000 - 平均奖励: 0.99 - 探索率: 0.01
回合 2000/2000 - 平均奖励: 0.99 - 探索率: 0.01

训练完成！

在 100 个测试回合中，成功次数: 100
成功率: 100.0%

最终的Q表:
[[0.73508445 0.77378094 0.6983373  0.73509183]
 [0.73509189 0.         0.         0.44689223]
 [0.44689223 0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.77377896 0.81450625 0.         0.73509189]
 [0.         0.         0.         0.